<a target="_blank" href="https://colab.research.google.com/github/sonder-art/fdd_o23/blob/main/codigo/polars/04_ejercicio_polars.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
! pip install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 127.2 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import polars as pl
import random
import string


# Generacion de Datos

In [4]:
# Funciones auxiliares para generar datos
def random_string(length=5):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for i in range(length))

def generate_data(n=1000):
    data = {
        "id_venta": list(range(1, n + 1)),
        "producto": [random_string() for _ in range(n)],
        "cantidad": [random.randint(1, 20) for _ in range(n)],
        "precio_unitario": [random.uniform(10, 100) for _ in range(n)],
        "fecha_venta": [pl.date(random.randint(2010, 2022), random.randint(1, 12), random.randint(1, 28)) for _ in range(n)],
        "cliente_id": [random.randint(1000, 5000) for _ in range(n)]
    }
    return pl.DataFrame(data)

# Generar el DataFrame
df = generate_data()
df.head()

id_venta,producto,cantidad,precio_unitario,fecha_venta,cliente_id
i64,str,i64,f64,object,i64
1,"""cbhjz""",10,86.267208,"2015-03-10 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1604
2,"""kbcry""",10,41.974531,"2015-09-08 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1574
3,"""hlxzu""",5,38.422662,"2019-09-14 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",1000
4,"""xcieo""",13,26.81343,"2016-06-21 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3437
5,"""vkecx""",10,47.448186,"2022-08-21 00:00:00.alias(""datetime"").strict_cast(Date).alias(""date"")",3817


In [5]:
df = df.lazy()
df

<LazyFrame [6 cols, {"id_venta": Int64 … "cliente_id": Int64}] at 0x7FCCDC3B6690>

# Limpieza de Datos

Utilizando Polars en modo Lazy, realice las siguientes operaciones de limpieza y manipulación de datos:

    1. `Filtrar` las ventas con cantidades menores a 5.
    2. `Agregar` una nueva columna llamada total_venta, que sea el resultado de multiplicar cantidad por precio_unitario.
    3. `Eliminar` la columna precio_unitario después de crear total_venta.
    4. `Agrupar` los datos por producto y calcular el total vendido y la cantidad total vendida por producto.
    5. `Ordenar` los resultados por el total vendido en orden descendente.

Todo esto lo debes de realizar en una sola linea de codigo

In [ ]:

lazy_df = df.lazy()

# Realizar las operaciones de limpieza y manipulación

resultado = (
    # Tu codigo aqui
    lazy_df
    
)

# Ejecutar y mostrar los resultados
resultado_df = resultado.collect()
print(resultado_df)


# Guarda el resultado en un archivo `parquet`

Guarda el resultado en un archivo parquet que se llame `resultados.parquet`

In [ ]:
# Tu codigo aqui

# Lee Parquet

Crea codigo para leer en polars el archivo parquet

In [ ]:
# Tu codigo aqui